In [ ]:
import pandas as pd
"""

Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений
"""
log = pd.read_csv('visit_log.csv', sep = ';')
# log.columns
log.head()
def source_type(row):
  type_ = ''
  if row['traffic_source'] in ['yandex', 'google']:
    type_ = 'organic'
  elif row['traffic_source'] in ['paid','email']:
    if row['region'] == 'Russia':
      type_ = 'ad'
    else:
      type_ = 'other'
  else:
    type_ = row['traffic_source']
  return type_

log['source_type'] = log.apply(source_type,axis = 1)
log.head()



,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [ ]:
"""
Задание 2
В файле URLs.txt содержатся url страниц новостного сайта. 
Вам необходимо отфильтровать его по адресам страниц 
с текстами новостей. Известно, 
что шаблон страницы новостей имеет внутри url следующую конструкцию:
 /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей,
 используя метод str.contains и регулярное выражение 
 в соответствии с заданным шаблоном
"""
df_url = pd.read_table('URLs.txt')
df_url.head()
len(df_url)

93

In [ ]:
df_url.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [ ]:
df__ = df_url[df_url.url.str.contains('/\d{8}-')]
df__.head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [ ]:
len(df__)

84

In [3]:
import pandas as pd

In [18]:
from pandas.core.tools.times import time
"""
Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. 
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 
Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.
"""
df_rating = pd.read_csv('ratings.csv')
df = df_rating.groupby('userId').count().reset_index()
# оценок больше 100
df_100 = df[df['rating'] > 100].reset_index()
# df_100.head()[['userId', 'movieId']]

# теперь с временем жизни
# оставлю без перевода, тк разница на то и разница
time_max = df_rating.groupby('userId').agg({'timestamp' : ['min','max']}).reset_index()
# time_max['delta'] = time_max['max'] - time_max['min']
time_max_delta = time_max[['userId','timestamp']].reset_index()
# time_max_delta.head()
time_max_delta['delta'] = time_max_delta['timestamp']['max'] - time_max_delta['timestamp']['min']
id_delta = time_max_delta.loc[:,['userId', 'delta']]
id_delta.head()

,userId,delta
,,
0,1,97
1,2,851
2,3,71198
3,4,203560
4,5,2101


In [19]:
id_100 = df_100[['userId']]
id_100.head(10)

,userId
0,4
1,8
2,15
3,17
4,19
5,21
6,22
7,23
8,26
9,30


In [20]:
last_df = id_100.merge(id_delta, how = 'left', on = 'userId').reset_index()
# delta в сек и хз почему такое странное название столбца
# где-то не хватило reset_index() ?
last_df

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:9203: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  validate=validate,


,index,userId,"(delta, )"
0,0,4,203560
1,1,8,85187
2,2,15,471393496
3,3,17,8053
4,4,19,5282
...,...,...,...
253,253,656,3053
254,254,659,31609411
255,255,664,98180439
256,256,665,54131471


In [21]:
last_df.reset_index().sum()

level_0            33153
index              33153
userId             88942
(delta, )    10340770922
dtype: int64

In [31]:
# не получается обратиться к '(delta, )'
# и так и не поняла почеу он так странно надывается после merge =(((

sum_sec = 10340770922 / len(last_df)
# JОТВЕТ: среднее время жизни в днях:
sum_day =  sum_sec / (24*60*60)
sum_day

463.8947621482917

In [ ]:
"""
Задание 4

Дана статистика услуг перевозок клиентов компании по типам 
(см. файл “Python_13_join.ipynb”
 в разделе Материалы для лекции «Продвинутый pandas»).

Необходимо сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id
без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов
"""

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [ ]:
# таблицу с тремя типами выручки для каждого client_id
# без указания адреса клиента
rzd_auto = rzd.merge(auto,how = 'outer', on='client_id')
rzd_auto_air = rzd_auto.merge(air, how='outer',on = 'client_id')
rzd_auto_air

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [ ]:
# аналогичную таблицу по
#  типам выручки с указанием адреса клиента
rzd_auto_air.merge(client_base, on='client_id',how='outer')

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4
